In [1]:
# ! conda create -y -n milvus python=3.10

! pip install pymilvus==2.4.0
! pip install langchain
! pip install ibm-generative-ai
! pip install unstructured
! pip install huggingface-hub
! pip install sentence-transformers
! pip install ibm_watsonx_ai


[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [2]:
import os
import requests
import json

from dotenv import load_dotenv

from langchain_community.document_loaders import UnstructuredURLLoader

from langchain.text_splitter import CharacterTextSplitter
from langchain_community.vectorstores import Milvus

from langchain.prompts import PromptTemplate
from langchain.callbacks.base import BaseCallbackHandler

# BAM
from genai.extensions.langchain import LangChainInterface
from genai.credentials import Credentials
from genai import Client

from genai.schema import TextGenerationParameters

from ibm_watsonx_ai.foundation_models import Embeddings

load_dotenv() 

False

In [3]:
token = ""
project_id = ""

In [4]:
text_splitter = CharacterTextSplitter(chunk_size=2000, chunk_overlap=0)
text_splitter.split_text("gogo")

['gogo']

In [5]:
embedding_model = Embeddings(
            model_id="ibm/slate-125m-english-rtrvr",
            credentials={"api_key": token, "url": "https://us-south.ml.cloud.ibm.com"},
            project_id=project_id,
            params={
                "truncate_input_tokens": 512
            }
        )
embedding_model

In [19]:
# INITIALIZE A MILVUS COLLECTION WITH EMBEDDINGS FROM A SET OF DOCUMENTS

urls = [
    "https://en.wikipedia.org/wiki/Silvio_Santos",
]

loader = UnstructuredURLLoader(urls=urls)
data = loader.load()

In [20]:

text_splitter = CharacterTextSplitter(chunk_size=2000, chunk_overlap=0)
docs = text_splitter.split_documents(data)

vector_db = Milvus.from_documents(
    docs,
    embedding_model,
    drop_old=True,  # Drop the old Milvus collection if it exists
    connection_args={"host":"823135eb-c10e-4ae5-9d93-62b881faa87b.cpltrlid0d8r2cjtm4e0.lakehouse.appdomain.cloud", 
                     "port":"31660", "secure":True,
                     "user":"ibmlhapikey", "password": token},
)

Created a chunk of size 14723, which is longer than the specified 2000


In [21]:
# TEST COLLECTION INITIALISATION WITH A SIMPLE SIMILARITY SEARCH

query = "Which year Silvio Santos step away from television?"
docs = vector_db.similarity_search(query)

docs

[Document(metadata={'source': 'https://en.wikipedia.org/wiki/Silvio_Santos', 'pk': 451797497425623813}, page_content="In 1999, Santos premiered Show do Milhão, a game show that was influenced by Who Wants to Be a Millionaire? (which would not have a local version until 2017, hosted by Luciano Huck for Globo) and become a major success for SBT through the early-2000s.[20][21] In 2001, Santos entered the reality television market by premiering Casa dos Artistas. The show was inspired by the Dutch format Big Brother; Santos had attempted to acquire the local rights to the format for SBT, but lost to Globo. The series was developed in secret, and only revealed on its premiere date of 28 October 2001—beating Globo's Big Brother Brasil to air, and drawing away viewers from Globo's highly-viewed newsmagazine Fantástico. Its similarities to Big Brother led to Globo briefly reaching an injunction to halt its broadcast, but Casa would later return to air. The first season was successful, with SB

In [22]:
# SET LANGCHAIN INTERFACE FOR GEN AI

creds = Credentials(token,
                    api_endpoint="https://us-south.ml.cloud.ibm.com") # credentials object to access the LLM service


params = TextGenerationParameters(decoding_method="sample",temperature=0.7, top_p=1, top_k=50, min_new_tokens=0, max_new_tokens=200
                        )

model = LangChainInterface(model_id= 'ibm/granite-13b-chat-v2', parameters=params, client=Client(credentials=creds))

prompt_template = (
            """[INST]
            <<SYS>>
                here are your instructions:
            1. You will be given a context that should be used to reply to user questions. The context will begin with \"BEGIN\" and will end with \"END\".
            2. You should generate the next response using informations available in the context.
            3. If you can't find an answer, say \"I don't know\".
            4. Do not use any other knowledge.
            5. Specify the sentences in context that justify your answer.
                BEGIN {context} END
            <</SYS>>
            <<HUMAN>>
              {question}
            <</HUMAN>>
            [/INST]
            """
        )

customprompt = PromptTemplate(
            template=prompt_template, input_variables=["context", "question"]
        )

In [23]:
model

LangChainInterface(client=<genai.client.Client object at 0x32439ca90>, model_id='ibm/granite-13b-chat-v2', parameters=TextGenerationParameters(beam_width=None, decoding_method='sample', include_stop_sequence=None, length_penalty=None, max_new_tokens=200, min_new_tokens=0, random_seed=None, repetition_penalty=None, return_options=None, stop_sequences=None, temperature=0.7, time_limit=None, top_k=50, top_p=1.0, truncate_input_tokens=None, typical_p=None))

In [24]:
def append_docs(docs):
    context = ''
    for doc in docs:
        context += doc.page_content
    return context

def get_wkc_token_on_cloud(apikey):
    body = {"grant_type": "urn:ibm:params:oauth:grant-type:apikey",
            "apikey": apikey}

    token_request = requests.post('https://iam.cloud.ibm.com/identity/token',
                                  data=body, headers={'content-type': 'application/x-www-form-urlencoded'})
    try:
        token_request.raise_for_status()
    except requests.exceptions.HTTPError as e:
        raise e  # return code not equals to 200

    return json.loads(token_request.text)["access_token"]


In [25]:
def qa(question):
    docs = vector_db.similarity_search(query)

    print(str(len(docs))+" nearest embeddings found;")
    
    context = append_docs(docs=docs)
    prompt = customprompt.format(context=context, question=question)

    barear_token = get_wkc_token_on_cloud(token)

    url = "https://us-south.ml.cloud.ibm.com/ml/v1/text/generation?version=2023-05-29"

    body = {
        "input": prompt,
        "parameters": {
            "decoding_method": "greedy",
            "max_new_tokens": 900,
            "repetition_penalty": 1.05
        },
        "model_id": "ibm/granite-13b-chat-v2",
        "project_id": project_id
    }

    headers = {
        "Accept": "application/json",
        "Content-Type": "application/json",
        "Authorization": "Bearer {}".format(barear_token)
    }

    response = requests.post(
        url,
        headers=headers,
        json=body
    )

    if response.status_code != 200:
        raise Exception("Non-200 response: " + str(response.text))
    
    data = response.json()
    
    return data['results'][0]['generated_text']

In [29]:
qa("How many siblings did Silvio Santos had?")

4 nearest embeddings found;


"I don't know. The context does not provide information about Silvio Santos's siblings."

In [30]:
qa("How much money did Silvio Santos had?")

4 nearest embeddings found;


'Santos had a net worth of US$3.2 billion.'

In [28]:
qa("Silvio Santos year of death?")

4 nearest embeddings found;


'Santos died on August 17, 2024.'